Deep Learning Models -- A collection of various deep learning architectures, models, and tips for TensorFlow and PyTorch in Jupyter Notebooks.
- Author: Sebastian Raschka
- GitHub Repository: https://github.com/rasbt/deeplearning-models

- Runs on CPU or GPU (if available)

# Model Zoo -- Multilayer Perceptron

## Imports

In [1]:
import time
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Settings and Dataset

In [2]:
##########################
### SETTINGS
##########################

# Device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 0

# Hyperparameters
random_seed = 1
learning_rate = 0.1
num_epochs = 10
batch_size = 64

# Architecture
num_features = 784
num_hidden_1 = 128
num_hidden_2 = 256
num_classes = 10


##########################
### MNIST DATASET
##########################

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


In [3]:
##########################
### MODEL
##########################

class MultilayerPerceptron(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(MultilayerPerceptron, self).__init__()
        
        ### 1st hidden layer
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        
        ### 2nd hidden layer
        self.linear_2 = torch.nn.Linear(num_hidden_1, num_hidden_2)
        
        ### Output layer
        self.linear_out = torch.nn.Linear(num_hidden_2, num_classes)
        
    def forward(self, x):
        out = self.linear_1(x)
        out = F.relu(out)
        out = self.linear_2(out)
        out = F.relu(out)
        logits = self.linear_out(out)
        probas = F.log_softmax(logits, dim=1)
        return logits, probas

    
torch.manual_seed(random_seed)
model = MultilayerPerceptron(num_features=num_features,
                             num_classes=num_classes)

model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [4]:
def compute_accuracy(net, data_loader):
    net.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.view(-1, 28*28).to(device)
            targets = targets.to(device)
            logits, probas = net(features)
            _, predicted_labels = torch.max(probas, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return correct_pred.float()/num_examples * 100
    

start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))

    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 000/938 | Cost: 2.3075
Epoch: 001/010 | Batch 050/938 | Cost: 1.7222
Epoch: 001/010 | Batch 100/938 | Cost: 0.7171
Epoch: 001/010 | Batch 150/938 | Cost: 0.5023
Epoch: 001/010 | Batch 200/938 | Cost: 0.3913
Epoch: 001/010 | Batch 250/938 | Cost: 0.4785
Epoch: 001/010 | Batch 300/938 | Cost: 0.4169
Epoch: 001/010 | Batch 350/938 | Cost: 0.1393
Epoch: 001/010 | Batch 400/938 | Cost: 0.2754
Epoch: 001/010 | Batch 450/938 | Cost: 0.3003
Epoch: 001/010 | Batch 500/938 | Cost: 0.3826
Epoch: 001/010 | Batch 550/938 | Cost: 0.2613
Epoch: 001/010 | Batch 600/938 | Cost: 0.2139
Epoch: 001/010 | Batch 650/938 | Cost: 0.3733
Epoch: 001/010 | Batch 700/938 | Cost: 0.1830
Epoch: 001/010 | Batch 750/938 | Cost: 0.2960
Epoch: 001/010 | Batch 800/938 | Cost: 0.1190
Epoch: 001/010 | Batch 850/938 | Cost: 0.1046
Epoch: 001/010 | Batch 900/938 | Cost: 0.3122
Epoch: 001/010 training accuracy: 92.79%
Time elapsed: 0.21 min
Epoch: 002/010 | Batch 000/938 | Cost: 0.2053
Epoch: 002/010 |

Epoch: 009/010 | Batch 750/938 | Cost: 0.0087
Epoch: 009/010 | Batch 800/938 | Cost: 0.0268
Epoch: 009/010 | Batch 850/938 | Cost: 0.0087
Epoch: 009/010 | Batch 900/938 | Cost: 0.0208
Epoch: 009/010 training accuracy: 99.07%
Time elapsed: 1.80 min
Epoch: 010/010 | Batch 000/938 | Cost: 0.0393
Epoch: 010/010 | Batch 050/938 | Cost: 0.0161
Epoch: 010/010 | Batch 100/938 | Cost: 0.0308
Epoch: 010/010 | Batch 150/938 | Cost: 0.0082
Epoch: 010/010 | Batch 200/938 | Cost: 0.0181
Epoch: 010/010 | Batch 250/938 | Cost: 0.0350
Epoch: 010/010 | Batch 300/938 | Cost: 0.0427
Epoch: 010/010 | Batch 350/938 | Cost: 0.0247
Epoch: 010/010 | Batch 400/938 | Cost: 0.0606
Epoch: 010/010 | Batch 450/938 | Cost: 0.0383
Epoch: 010/010 | Batch 500/938 | Cost: 0.0640
Epoch: 010/010 | Batch 550/938 | Cost: 0.0871
Epoch: 010/010 | Batch 600/938 | Cost: 0.1112
Epoch: 010/010 | Batch 650/938 | Cost: 0.0295
Epoch: 010/010 | Batch 700/938 | Cost: 0.0333
Epoch: 010/010 | Batch 750/938 | Cost: 0.0375
Epoch: 010/010 |

In [5]:
%%time
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 97.76%
CPU times: user 892 ms, sys: 12 ms, total: 904 ms
Wall time: 901 ms
